## Transformer

模型结构：
![pic1](https://ws2.sinaimg.cn/large/006tKfTcly1g1bkwxdi7lj30k00fcdgu.jpg)
和大多数seq2seq模型一样，transformer的结构也是由encoder和decoder组成。

Encoder由N=6个相同的layer组成，layer指的就是上图左侧的单元，最左边有个“Nx”，这里是x6个。每个Layer由两个sub-layer组成，分别是multi-head self-attention mechanism和fully connected feed-forward network。其中每个sub-layer都加了residual connection和normalisation，因此可以将sub-layer的输出表示为：
![pic2](https://ws2.sinaimg.cn/large/006tKfTcly1g1bkzs6g7tj30go014747.jpg)

接下来按顺序解释一下这两个sub-layer：

Multi-head self-attention:
attention可由以下形式表示：
![pic3](https://ws4.sinaimg.cn/large/006tKfTcly1g1bl07kfmdj30go014t8l.jpg)

multi-head attention则是通过h个不同的线性变换对Q，K，V进行投影，最后将不同的attention结果拼接起来：
![pic4](https://ws2.sinaimg.cn/large/006tKfTcly1g1bl0nu01sj30go017wef.jpg)
![pic5](https://ws2.sinaimg.cn/large/006tKfTcly1g1bl134y5bj30go018jra.jpg)

self-attention则是取Q，K，V相同。

另外，文章中attention的计算采用了scaled dot-product，即：
![pic6](https://ws2.sinaimg.cn/large/006tKfTcly1g1bl19xhfzj30go01s3yf.jpg)

作者同样提到了另一种复杂度相似但计算方法additive attention，在$d_{k}$很小的时候和dot-product结果相似,$d_{k}$大的时候，如果不进行缩放则表现更好，但dot-product的计算速度更快，进行缩放后可减少影响（由于softmax使梯度过小，具体可见论文中的引用）。


Decoder和Encoder的结构差不多，但是多了一个attention的sub-layer，这里先明确一下decoder的输入输出和解码过程：

输出：对应i位置的输出词的概率分布
输入：encoder的输出 & 对应i-1位置decoder的输出。所以中间的attention不是self-attention，它的K，V来自encoder，Q来自上一位置decoder的输出
解码：这里要特别注意一下，编码可以并行计算，一次性全部encoding出来，但解码不是一次把所有序列解出来的，而是像rnn一样一个一个解出来的，因为要用上一个位置的输入当作attention的query
明确了解码过程之后最上面的图就很好懂了，这里主要的不同就是新加的另外要说一下新加的attention多加了一个mask，因为训练时的output都是ground truth，这样可以确保预测第i个位置时不会接触到未来的信息。

加了mask的attention原理如图（另附multi-head attention）：
![pic7](https://ws2.sinaimg.cn/large/006tKfTcly1g1bl6p5dtmj30k00920t8.jpg)


## BERT

BERT的全称是Bidirectional Encoder Representation from Transformers，即双向Transformer的Encoder，因为decoder是不能获要预测的信息的。模型的主要创新点都在pre-train方法上，即用了Masked LM和Next Sentence Prediction两种方法分别捕捉词语和句子级别的representation。

模型结构：
![pic8](https://ws2.sinaimg.cn/large/006tKfTcly1g1blaec55sj30k004yq3w.jpg)

对比OpenAI GPT(Generative pre-trained transformer)，BERT是双向的Transformer block连接；就像单向rnn和双向rnn的区别，直觉上来讲效果会好一些。

对比ELMo，虽然都是“双向”，但目标函数其实是不同的。ELMo是分别以P($w_{i}$|$w_{1}$, ...$w_{i-1}$) 和 P($w_{i}$|$w_{i+1}$, ...$w_{n}$) 作为目标函数，独立训练处两个representation然后拼接，而BERT则是以 P($w_{i}$|$w_{1}$,  ...,$w_{i-1}$, $w_{i+1}$,...,$w_{n}$) 作为目标函数训练LM。

### Embedding

这里的Embedding由三种Embedding求和而成：
![pic9](https://ws2.sinaimg.cn/large/006tKfTcly1g1blaec55sj30k004yq3w.jpg)

Token Embeddings是词向量，第一个单词是CLS标志，可以用于之后的分类任务;
Segment Embeddings用来区别两种句子，因为预训练不光做LM还要做以两个句子为输入的分类任务;
Position Embeddings和之前文章中的Transformer不一样，不是三角函数而是学习出来的.

### Fine-Tunning

![pic10](https://ws4.sinaimg.cn/large/006tKfTcly1g1blg4h2apj30k00k2tbd.jpg)

可以调整的参数和取值范围有：
Batch size: 16, 32
Learning rate (Adam): 5e-5, 3e-5, 2e-5
Number of epochs: 3, 4

### 优点
BERT是截至2018年10月的最新state of the art模型，通过预训练和精调横扫了11项NLP任务，这首先就是最大的优点了。而且它还用的是Transformer，也就是相对rnn更加高效、能捕捉更长距离的依赖。对比起之前的预训练模型，它捕捉到的是真正意义上的bidirectional context信息。

### 缺点
作者在文中主要提到的就是MLM预训练时的mask问题：

[MASK]标记在实际预测中不会出现，训练时用过多[MASK]影响模型表现
每个batch只有15%的token被预测，所以BERT收敛得比left-to-right模型要慢（它们会预测每个token）